In [1]:
import gc
import sys
import time
sys.path.append("/home/shaow/jupyter_base/pycharm_001")
base = "/home/shaow/jupyter_base/pycharm_001"
import torch

import graph_process as G
import obj.GraphBase as GB
import utils

appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge, \
           id_index_map ,index_id_map, index_keyword_map, index_ros_map, index_app_map , \
           app_dict = G.process_year_graph(2019)



gb_2019 = GB.GraphBase(appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge,
           id_index_map, index_id_map, index_ros_map, index_app_map, index_keyword_map ,
           app_dict)


8.0.19
10
socket 10.0.82.237:3306
done init 3564 nodes from apply id
244800
done init 593205 nodes from application


In [38]:
# prepare F data and edge data
gb = gb_2019.get_split_appid_map('F')
t_data = G.get_graph_edges(gb)


(2, 795)
<class 'int'>
<class 'numpy.int64'>
(2, 24467)
<class 'int'>
<class 'numpy.int64'>
(2, 24467)
<class 'int'>
<class 'numpy.int64'>
(2, 111638)
<class 'int'>
<class 'numpy.int64'>


In [39]:
train_data = t_data.train
valid_data = t_data.validation
test_data = t_data.test
# num_rels = t_data.num_rels
num_nodes = gb.node_count()
num_rels = gb.rel_count()

In [51]:
import importlib
import experiment.model.inverseModel.ConvE as ConvE
importlib.reload(ConvE)
importlib.reload(utils)


<module 'utils' from '/home/shaow/jupyter_base/pycharm_001/utils.py'>

In [82]:
test_data = torch.from_numpy(test_data)
use_cuda = torch.cuda.is_available()

TypeError: expected np.ndarray (got Tensor)

In [83]:
if use_cuda:
    torch.cuda.set_device('cuda:0')

In [84]:
forward_time = []
backward_time = []
epochs = 6000
label_smooth = 0.001
model_path = "convE.pth"
best_mrr = 0

In [96]:
model = ConvE.ConvE(num_entities=num_nodes, num_relations=num_rels, embedding_dim=200, embedding_shape1=20, hidden_size=9728)
opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.005)

79178 5


In [ ]:
import numpy as np
if use_cuda:
    model.cuda()
adj_list, degrees = utils.get_adj_and_degrees(num_nodes, train_data)
src, rel, dst = train_data.transpose()
uniq_v, _ = np.unique((src, dst), return_inverse=True)
node_id = torch.from_numpy(uniq_v).view(-1, 1).long()
triple = torch.from_numpy(train_data)
if use_cuda:
    node_id = node_id.cuda()
    test_data = test_data.cuda()
    triple = triple.cuda()
e1 = triple[:,0]
rel = triple[:,1]
e2 = triple[:,2]

Epoch 0000 | Loss 119.2927 | Best MRR 0.0002 | Forward 8.2523s | Backward 0.0000s
MRR (raw): 0.000128
Hits (raw) @ 1: 0.000000
Hits (raw) @ 3: 0.000031
Hits (raw) @ 10: 0.000093
Epoch 0001 | Loss 50.5936 | Best MRR 0.0002 | Forward 8.9955s | Backward 0.0000s
Epoch 0002 | Loss 53.3434 | Best MRR 0.0002 | Forward 9.4829s | Backward 0.0000s
MRR (raw): 0.000141
Hits (raw) @ 1: 0.000031
Hits (raw) @ 3: 0.000031
Hits (raw) @ 10: 0.000062
Epoch 0003 | Loss 58.1285 | Best MRR 0.0002 | Forward 8.9747s | Backward 0.0000s
Epoch 0004 | Loss 51.8350 | Best MRR 0.0002 | Forward 9.6069s | Backward 0.0000s
MRR (raw): 0.000118
Hits (raw) @ 1: 0.000000
Hits (raw) @ 3: 0.000000
Hits (raw) @ 10: 0.000093
Epoch 0005 | Loss 53.8302 | Best MRR 0.0002 | Forward 9.5072s | Backward 0.0000s
Epoch 0006 | Loss 56.5126 | Best MRR 0.0002 | Forward 9.5902s | Backward 0.0000s
MRR (raw): 0.000136
Hits (raw) @ 1: 0.000000
Hits (raw) @ 3: 0.000031
Hits (raw) @ 10: 0.000185
Epoch 0007 | Loss 53.0348 | Best MRR 0.0002 | Fo

In [ ]:
for epoch in range(epochs):
        model.train()
        t0 = time.time()
        batch_size = 1024
        test_size = num_nodes
        n_batch = (test_size + batch_size - 1) // batch_size
        train_loss = 0.
        for idx in range(n_batch):
        # print("batch {} / {}".format(idx, n_batch))
            batch_start = idx * batch_size
            batch_end = min(test_size, (idx + 1) * batch_size)
            batch_a = e1[batch_start: batch_end]
            batch_r = rel[batch_start: batch_end]
            this_batch_size = batch_end - batch_start
            e2_multi = torch.zeros(this_batch_size, num_nodes)
            if use_cuda :
                e2_multi = e2_multi.cuda()
            e2_multi = e2_multi.scatter_(1, e2[batch_start: batch_end].reshape(this_batch_size, 1), 1).float()
            e2_multi = ((1.0 - label_smooth)*e2_multi) + (1.0/e2_multi.size(1))

            pred = model(batch_a, batch_r)
            loss = model.loss(pred, e2_multi)
            # print(loss)
            train_loss += loss.cpu().data
            loss.backward()
            opt.step()
        t1 = time.time()
        t2 = time.time()
        forward_time.append(t1 - t0)
        backward_time.append(t2 - t1)
        print("Epoch {:04d} | Loss {:.4f} | Best MRR {:.4f} | Forward {:.4f}s | Backward {:.4f}s".
                format(epoch, train_loss, best_mrr, forward_time[-1], backward_time[-1]))
        opt.zero_grad()
        if epoch % 2 == 0:
            model.eval()
            # test_data
            mrr = ConvE.calc_mrr(model, test_data,
                             hits=[1, 3, 10], eval_bz=500)
            if mrr > best_mrr:
                best_mrr = mrr
                print('saving to {0}'.format(model_path))
                torch.save({'state_dict': model.state_dict(), 'epoch': epoch},
                           model_path)

In [2]:
from gensim.models import word2vec
semantic_base = word2vec.Word2Vec.load('2019_model')
